In [1]:
!pip install keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 1.2 MB/s eta 0:00:00


In [24]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    roc_auc_score
)

# =========================
# 1. LOAD DATA
# =========================
df = pd.read_csv("churn_preprocessed.csv")

X = df.drop("Exited", axis=1)
y = df["Exited"]

# Same split as training
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# =========================
# 2. LOAD MODEL
# =========================
model = load_model("churn_model_best.h5")

# =========================
# 3. PREDICTIONS
# =========================
y_prob = model.predict(X_test)        # probabilities
y_pred = (y_prob > 0.5).astype(int)   # binary output

# =========================
# 4. EVALUATION METRICS
# =========================
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_prob)

# =========================
# 5. PRINT RESULTS
# =========================
print("✅ Model Evaluation Results\n")
print(f"Accuracy      : {accuracy * 100:.2f}%\n")
print("Confusion Matrix:")
print(cm, "\n")
print("Classification Report:")
print(report)
print(f"ROC-AUC Score : {roc_auc:.4f}")


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
✅ Model Evaluation Results

Accuracy      : 85.90%

Confusion Matrix:
[[1542   65]
 [ 217  176]] 

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1607
           1       0.73      0.45      0.56       393

    accuracy                           0.86      2000
   macro avg       0.80      0.70      0.74      2000
weighted avg       0.85      0.86      0.85      2000

ROC-AUC Score : 0.8532


In [26]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model


class ChurnPipeline:
    def __init__(self):
        print("🔹 Loading model and feature structure...")

        # Load trained model
        self.model = load_model("churn_model_best.h5")

        # Load feature structure from preprocessed data
        df = pd.read_csv("churn_preprocessed.csv")
        self.feature_columns = df.drop("Exited", axis=1).columns

        print("✅ Pipeline ready for prediction")

    def predict(self, input_dict):
        """
        input_dict: dictionary of feature values
        """

        # Convert input to DataFrame
        input_df = pd.DataFrame([input_dict])

        # Ensure correct column order
        input_df = input_df[self.feature_columns]

        # Prediction
        prob = self.model.predict(input_df)[0][0]

        if prob > 0.5:
            return "❌ Customer WILL CHURN"
        else:
            return "✅ Customer will NOT churn"


In [27]:
pipeline = ChurnPipeline()

sample_input = {
    "CreditScore": 600,
    "Gender": 1,             # already encoded
    "Age": 40,
    "Tenure": 3,
    "Balance": 60000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000,
    "Geography_Germany": 0,
    "Geography_Spain": 1
}

print(pipeline.predict(sample_input))


🔹 Loading model and feature structure...
✅ Pipeline ready for prediction
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
❌ Customer WILL CHURN
